In [2]:
# This code is used to store the ML models used for the web app

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
     
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import json
import os
import re

import gensim 
from gensim.models import Word2Vec
import gensim.downloader

import pickle 

import nltk
from nltk.tokenize import word_tokenize

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
tqdm.pandas()

In [4]:
import sys
sys.path.append('../src')

In [5]:
# call method
from methods import feature_extraction, evaluation

# Tactic Model: 

## Open Files: 

In [6]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

In [7]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Classifiers:

In [8]:
mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, Y_train)
predictions_mlp = mlp.predict(X_test)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
evaluation(predictions_mlp, Y_test)

,metric,result
0,macro precision,0.738212
1,micro precision,0.735568
2,macro recall,0.597994
3,micro recall,0.645953
4,macro fscore,0.696951
5,micro fscore,0.715709


In [10]:
# Save as pickle: 

with open('/homes/lgf21/API/app/tactic_MLPCV_model.pickle', 'wb') as handle:
    pickle.dump(mlp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [9]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [10]:
evaluation(Y_pred, Y_test)

,metric,result
0,macro precision,0.668056
1,micro precision,0.704846
2,macro recall,0.630060
3,micro recall,0.654129
4,macro fscore,0.658896
5,micro fscore,0.694083


In [10]:
# Save as pickle: 

with open('tactic_SVmodel.pickle', 'wb') as handle:
    pickle.dump(sv_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
len(sv_classifier.feature_names_in_)

73887

In [28]:
# Save as pickle: 

with open('tactic_train_test.pickle', 'wb') as handle:
    pickle.dump((X_train, X_test, Y_train, Y_test), handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# Technique Model:

In [6]:
#Open Pickle without lemmatisation for techniques: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, _, _, Y_train, Y_test) = pickle.load(handle)

In [7]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(SVC(max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


KeyboardInterrupt: 

In [ ]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [ ]:
evaluation(Y_pred, Y_test)

In [11]:
# Save as pickle: 

with open('technique_SVModel.pickle', 'wb') as handle:
    pickle.dump((sv_classifier), handle, protocol=pickle.HIGHEST_PROTOCOL)